# Dogs vs Cats Image Classification Without Image Augmentation
## Import Packages

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Data Loading

In [4]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

68608000/68606236 [==============================] - 18s 0us/step


In [5]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

/home/ritesh/.keras/datasets
/home/ritesh/.keras/datasets/fashion-mnist
/home/ritesh/.keras/datasets/cats_and_dogs_filtered
/home/ritesh/.keras/datasets/cats_and_dogs_filtered/train
/home/ritesh/.keras/datasets/cats_and_dogs_filtered/train/dogs
/home/ritesh/.keras/datasets/cats_and_dogs_filtered/train/cats
/home/ritesh/.keras/datasets/cats_and_dogs_filtered/validation
/home/ritesh/.keras/datasets/cats_and_dogs_filtered/validation/dogs
/home/ritesh/.keras/datasets/cats_and_dogs_filtered/validation/cats


In [6]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

## Understanding our data 

In [7]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val